In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.chdir('..')

In [2]:
# os.environ.get("ZHIPUAI_API_KEY")

# WritingProject

## 每一步都询问：使用默认的智谱AI

In [3]:
from langchain_chinese import WritingTask

# 使用默认的智谱AI推理
wp = WritingTask(task_mode="askme")

Bug清单：
- [ ] START 记忆确实存在，但无法通过 START@memory 正确提取
- [ ] ask 指令无法在当前节点重新生成
- [ ] todos清单增加一个标记为当前任务的状态：当前任务｜未完成｜已完成
- [ ] all列表中应当增加 START 和 END

In [4]:
wp.run("致孩子的一封信, 800字", max_steps=1)

[START]
ask AI: 致孩子的一封信, 800字
[ChatPromptTemplate(input_variables=['history', 'task'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'task_instruction': '你是一名优秀的写手，任务是对写作任务做评估，给出总体写作建议。', 'output_format': '\n（请确保输出符合JSON语法限定，并且不要出现其他的JSON键，以便我能够正确解析）\n```json\n{\n    "总字数要求": [int类型]预计的总体字数要求，默认为1000字,\n    "标题名称": [str类型]根据写作任务，给出用户要求或你推荐的标题名称，不要带编号,\n    "扩写指南": [str类型]应当尽量包含写作任务中提及的写作要求，也可以包含你的创作建议中所涉及的人物、地点、情节等实体名称和背景设定\n}\n```\n', 'json_instruction': '1. 你只能输出一个JSON段落，否则我将无法正确解析。2. 你必须严格遵循我提出的JSON键值规则，不要额外发挥，否则我将无法正确解析。3. 如果你的创作中出现实体名称、创作设定等，就将其单独提炼到扩写指南或内容摘要；这样做非常必要，可以让独立的创作子任务保持一致的背景设定。'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['json_instruction', 'output_form

In [5]:
wp.run("ok", max_steps=1)

[START]
-------------------- TODOs --------------------
* <0> 800字以内 | 《致孩子的一封信：爱的絮语》
ask AI: 请开始！
[ChatPromptTemplate(input_variables=['history', 'task'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'words_limit': 500, 'words_advice': 800, 'title': '致孩子的一封信：爱的絮语', 'outline_exist': '《致孩子的一封信：爱的絮语》\n总字数要求约800字；* 未完成。\n扩写指南 >>> 本文是一封父母给予孩子的信，应包含深切的情感、生活经验的分享以及对孩子未来的期许。信中可以提及具体的家庭故事、父母的感悟以及对孩子的鼓励。建议在信中采用第一人称，使内容更加亲切和个人化。\n\n', 'task_instruction': '你是一名优秀的写手，可以构思写作思路、扩展写作提纲。', 'howto': '本文是一封父母给予孩子的信，应包含深切的情感、生活经验的分享以及对孩子未来的期许。信中可以提及具体的家庭故事、父母的感悟以及对孩子的鼓励。建议在信中采用第一人称，使内容更加亲切和个人化。', 'output_format': '\n（请确保大纲列表中至少两个子项；确保输出符合JSON语法限定，并且不要出现其他的JSON键，以便我能够正确解析）\n```json\n{\n    "大纲列表": [\n        {\n           

In [6]:
wp.run("title", max_steps=1)

[0]
致孩子的一封信：爱的絮语


In [7]:
wp.run("ok", max_steps=1)

[0]
-------------------- TODOs --------------------
* <1> 400字以内 | 《爱的回忆》
* <2> 400字以内 | 《未来的期许》
ask AI: 请开始！
[ChatPromptTemplate(input_variables=['history', 'task'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'words_limit': 500, 'words_advice': 400, 'title': '爱的回忆', 'outline_exist': '《致孩子的一封信：爱的絮语》\n总字数要求约800字；* 未完成。\n扩写指南 >>> 本文是一封父母给予孩子的信，应包含深切的情感、生活经验的分享以及对孩子未来的期许。信中可以提及具体的家庭故事、父母的感悟以及对孩子的鼓励。建议在信中采用第一人称，使内容更加亲切和个人化。\n\n1 爱的回忆 \n  扩写指南 >>> 回忆与孩子共同度过的美好时光，如一次家庭旅行、一个孩子成长的瞬间等，传递出父母对孩子的深情和怀念。\n  内容摘要 >>> None\n2 未来的期许 \n  扩写指南 >>> 表达父母对孩子的未来期望和鼓励，可以包括希望孩子具备的品质、实现的目标以及父母愿意提供的支持。\n  内容摘要 >>> None', 'task_instruction': '你是一名优秀的写手，负责详细构思段落细节。', 'howto': '回忆与孩子共同度过的美好时光，如一次家庭旅行、一个孩子成长的瞬间等，传递出父母对孩子的深情和怀念。', 

In [ ]:
wp.run()

In [9]:
wp.get_memory("START")

[HumanMessage(content='致孩子的一封信, 800字'),
 AIMessageChunk(content='```json\n{\n    "总字数要求": 800,\n    "标题名称": "致孩子的一封信",\n    "扩写指南": "本文是一封父母给予孩子的信，信中应包含关爱、期望、人生经验分享等内容。建议在写作中提及孩子的名字，以及一些家庭特有的记忆和事件，使信件更具个性和亲切感。"\n}\n```', response_metadata={'finish_reason': 'stop', 'model': 'glm-4', 'created': 1715419842, 'index': 0, 'usage': {'prompt_tokens': 267, 'completion_tokens': 83, 'total_tokens': 350}}, id='8635249280228058163')]

In [15]:
wp.get_memory(0)

[HumanMessage(content='请开始！'),
 AIMessageChunk(content='```json\n{\n    "大纲列表": [\n        {\n            "总字数要求": 400,\n            "标题名称": "温暖的问候与回忆",\n            "扩写指南": "以亲切的口吻问候孩子，回忆一些孩子童年时期的趣事，如初次骑自行车、家庭旅行等，传递家庭的温暖和爱。"\n        },\n        {\n            "总字数要求": 400,\n            "标题名称": "人生的智慧与期望",\n            "扩写指南": "分享一些人生经验和教训，如诚信、努力的重要性，并对孩子的未来表达美好的期望和鼓励，强调无论遇到什么困难，家人都是孩子坚强的后盾。"\n        }\n    ]\n}\n```', response_metadata={'finish_reason': 'stop', 'model': 'glm-4', 'created': 1715419849, 'index': 0, 'usage': {'prompt_tokens': 471, 'completion_tokens': 147, 'total_tokens': 618}}, id='8635248111996859323')]

In [7]:
wp.print_all()

-------------------- All --------------------
  <root> 1000字以内 | 《深情的告白》
  <0000001.10> 250字以内 | 《你的善良与光芒》
* <0000002.46> 250字以内 | 《共度的美好时光》
* <0000003.87> 250字以内 | 《你的才华与激情》
* <0000004.97> 250字以内 | 《未来生活的期许》


## 全自动生成：使用默认的智谱AI

In [3]:
from langchain_chinese import WritingTask

# 使用默认的智谱AI推理
# 你需要准备好智谱AI的APIKEY
wp = WritingTask(auto_mode="all")
wp.run("我今天扒了二牛的裤子，然后他一整天都不理我，帮我写一封1000字道歉信，我希望和他继续做好朋友，每段不要少于500字哦")

[START]
```json
{
    "总字数要求": 1000,
    "标题名称": "致二牛的诚挚道歉信",
    "扩写指南": "本信旨在对扒裤子事件进行深刻反省，表达诚挚的歉意，并请求二牛的谅解。信中需详细描述事件经过、自身错误、对二牛的影响以及改过自新的决心。同时，强调珍惜友谊，希望重修旧好。"
}
```
-------------------- TODOs --------------------
* [0] 约1000字 | 《致二牛的诚挚道歉信》
```json
{
    "大纲列表": [
        {
            "总字数要求": 500,
            "标题名称": "事件经过与自我反省",
            "扩写指南": "详细描述扒裤子事件的经过，深刻反省自己的错误，承认对二牛造成的尴尬和伤害。"
        },
        {
            "总字数要求": 500,
            "标题名称": "诚挚道歉与改过决心",
            "扩写指南": "表达对二牛的诚挚歉意，承诺采取具体措施改正错误，并展现对两人友谊的珍视及希望恢复关系的决心。"
        }
    ]
}
```
-------------------- TODOs --------------------
* [1] 约500字 | 《事件经过与自我反省》
* <2> 约500字 | 《诚挚道歉与改过决心》
```json
{
    "详细内容": "几天前，在学校的操场上，我和几个朋友玩闹之间，一时冲动地扒下了二牛的裤子。事情发生得太快，我甚至没有意识到这一举动会带来什么样的后果。现在回想起来，我深刻地认识到这一行为是多么的无礼和不当。我不仅让二牛在众人面前尴尬无比，更是侵犯了他的尊严和隐私。我对此感到非常抱歉，我知道我的错误给二牛带来了心理上的伤害，我真的很后悔。",
    "内容摘要": "描述了扒裤子事件的经过，并进行了自我反省，承认错误给二牛带来的尴尬和伤害。"
}
```
-------------------- TODOs --------------------
* [2] 约500字 | 《诚挚道歉与改过决心》
```json
{
  "详细

In [6]:
x = """
1 事件经过
 那是一个玩笑过火的下午，我们一群朋友在公园里嬉闹，我因一时的顽皮与冲动，竟然在众目睽睽之下扒下了二牛的裤子。当时，我只是想着逗大家笑，却完全忽略了二牛的面子和感受。笑声中，我看到了二牛的尴尬和愤怒，那一刻，我突然意识到自己的行为是多么的愚蠢和错误。
2 自我反省
 在深入分析自己的行为背后的原因时，我意识到那天的举动源于一种愚蠢的冲动和对自己幽默感的过度自信。我一直以为，作为朋友，我们可以毫无顾忌地开玩笑。然而，我忽视了你的感受，更忽略了这样一个事实：真正的幽默绝不会以牺牲他人的尊严为代价。我现在深刻认识到，那天的行为不仅令你尴尬，更伤害了你的自尊。对此，我感到无比懊悔。
3 对二牛的歉意
 二牛，我想借此机会，用我最真诚的心意向你道歉。我深刻意识到在那次公园的尴尬事件中，我扒下你的裤子，给你带来了多大的伤害和不适。这不仅是一个简单的恶作剧，更是对你个人尊严的侵犯。我承认，这一行为严重影响了我们之间的友谊，我对此感到万分抱歉。我完全理解你在那一刻的感受，以及这件事可能给你带来的长期影响。我后悔莫及，如果时光可以倒流，我绝不会让那样的场面发生。
4 友谊的重要性
 二牛，自从公园那件事发生后，我一直在反思，你的友谊对我意味着什么。它不仅是快乐时光的分享，也是我在逆境中的支撑。我意识到，那天我的举动不仅仅是一个玩笑，它深深地伤害了你，也动摇了我们之间的信任基石。你对我来说非常重要，我珍视我们的友谊，它让我的生活更加丰富多彩。因此，我愿意不遗余力地去修复这段关系，证明我对这段友谊的重视和诚意。
5 改善关系的具体行动
 为了表达我的诚意和决心，我计划采取以下具体行动来改善我们的关系：首先，我会在第一时间私下向你道歉，面对面的交流可以让我更直接地传达我的悔意。其次，我会在我们共同的朋友圈中公开澄清这一事件的真相，说明这是我个人不当的行为，并非有意为之，以消除可能给你带来的负面影响。此外，我将改正自己的行为，承诺今后在公共场合保持适当的举止，不再让类似的事情发生。我还会主动参与一些能体现我对这段友谊重视的活动，比如一起参加你感兴趣的活动，或是在你需要帮助时伸出援手。通过这些具体的行动，我希望能够证明我对于这段友谊的珍视，以及对于过去错误行为的深刻反省。"""

len(x)

946

In [10]:
wp.print_text()

《致二牛的诚挚道歉信》
总字数要求约2000字；已完成。
扩写指南 >>> 信中需要深刻反省自己的不当行为，表达诚挚的歉意，并说明为何希望维持与二牛的友谊。分段建议：首段表达歉意及反省；次段详细描述事件经过及对二牛的影响；第三段阐述自己的感悟及友谊的重要性；末段再次表达歉意并期望得到二牛的谅解。涉及人物：我，二牛。事件背景：一次不当的玩笑导致二牛的不悦。

1 深表歉意，反省自我
 二牛，我在此深感愧疚地向你道歉。我必须承认，我之前的行为——在那个不合时宜的场合下开的玩笑，特别是扒裤子的行为——是对你极大的不尊重。我现在回想起来，深感后悔和自责。那是一个极度缺乏考虑的行为，我完全忽略了它可能给你带来的尴尬和伤害。我为自己的无理和轻率感到羞愧，我真诚地为这次愚蠢的行为向你道歉。
2 事件经过及对二牛的影响
 那天的聚会，在大家兴致高昂时，我错误地认为扒掉二牛的裤子只是一个无伤大雅的玩笑。然而，当二牛的尴尬和惊慌显现在脸上时，我才意识到自己的行为是多么的过分。那一刻，笑声变得刺耳，二牛的尊严在我轻率的举动中受损。我完全能理解，对于向来内向的二牛来说，这样的公开羞辱是何等的打击，它不仅造成了短暂的尴尬，更在二牛心中留下了深刻的伤害。
3 感悟友谊，珍视情感
 这次扒裤子的玩笑事件，让我深刻地意识到了自己的不成熟和对友谊的无知。二牛，你的尴尬和痛苦，我虽无法完全感同身受，但我看到了你的沉默和眼中的失落。这让我明白，真正的友谊不是建立在取笑和伤害之上，而是需要相互尊重和理解。我珍视与你的友情，它如同生活中的清泉，给予我支持和力量。未来，我会用行动来证明我的改变：无论是在你的需要时伸出援手，还是在日常生活中尊重你的感受，我都会努力成为一个更好的朋友，以此来弥补这次的过错。
4 再次致歉，期待谅解
 再次致歉，二牛，我深知我的行为已经严重伤害了你的情感，我对此感到万分抱歉。我承认，那天的玩笑太过分了，完全没有考虑到你的感受。我后悔不已，如果可以，我愿意回到那一刻，收回我的举动。我们的友谊对我来说无比珍贵，我不希望因为这次愚蠢的错误而失去它。我诚恳地请求你的原谅，希望你能给我们之间的友情一个机会，让它能够经受住这次考验，继续走下去。


In [7]:
wp.get_memory("2")

[HumanMessage(content='请开始！'),
 AIMessageChunk(content='```json\n{\n    "详细内容": "那天的聚会，在大家兴致高昂时，我错误地认为扒掉二牛的裤子只是一个无伤大雅的玩笑。然而，当二牛的尴尬和惊慌显现在脸上时，我才意识到自己的行为是多么的过分。那一刻，笑声变得刺耳，二牛的尊严在我轻率的举动中受损。我完全能理解，对于向来内向的二牛来说，这样的公开羞辱是何等的打击，它不仅造成了短暂的尴尬，更在二牛心中留下了深刻的伤害。",\n    "内容摘要": "描述了在聚会中扒掉二牛裤子的经过，以及这一行为对二牛的内心造成的尴尬和伤害，表达了对二牛心情的理解和同情。涉及人物：我，二牛。事件背景：在聚会上的一次不恰当玩笑给二牛带来尴尬和伤害。"\n}\n```', response_metadata={'finish_reason': 'stop', 'model': 'glm-4', 'created': 1715352031, 'index': 0, 'usage': {'prompt_tokens': 662, 'completion_tokens': 187, 'total_tokens': 849}}, id='8635244263706067754')]

## 查看记忆和对话历史

In [26]:
[k for k in wp.memory._shorterm_memory_store]

['root',
 '20240508.180514.00001.408',
 '20240508.180514.00002.626',
 (('session_id', 'root'),),
 (('session_id', '20240508.180514.00002.626'),)]

In [27]:
for x in wp.memory.get_shorterm_memory(session_id="20240508.180514.00002.626").chat_memory.messages:
    print("-"*40)
    print(x.content)

----------------------------------------
请帮我扩写《与生物的互动》, 字数不超过500字，扩写依据为：详述主人公在山涧遇到的奇特生物，他们的互动过程，以及主人公如何凭借勇敢和智慧与生物相处，并从中得到的感悟。
----------------------------------------
```json
{
    "类型": "paragraph",
    "标题名称": "与生物的互动",
    "详细内容": "李明在山涧边遇见了一种他从未见过的生物，它们有着透明的翅膀，身体呈彩虹色，被水珠沾湿后闪烁着光芒。这些生物似乎并不惧怕人类，反而对李明的到来充满好奇。起初，李明有些害怕，担心这些奇特生物可能会有危险。但他很快冷静下来，细心观察它们的行为，试图理解它们。通过模仿它们的动作，他逐渐与这些生物建立起一种特殊的沟通方式。他发现，当自己展现出平静和善意时，这些生物也会给予他同样的回应。在互动的过程中，李明意识到，勇敢不仅仅是面对恐惧，更是在未知面前保持冷静和理智。而智慧则是通过观察和理解，去适应和融入周围的环境。",
    "内容摘要": "在山涧的奇遇中，李明遇到了一群奇特的彩虹色生物，通过勇敢和智慧的互动，他学会了与自然和谐相处的重要性，理解了勇敢和智慧在适应环境中的意义。"
}
```
----------------------------------------
字多些，1000字吧
----------------------------------------
```json
{
    "类型": "paragraph",
    "标题名称": "山涧奇遇",
    "详细内容": "李明在神秘的山林中迷路了。起初，他焦虑不安，四周是茂密的树林和蜿蜒的小径，让他辨不清方向。但就在他几乎要放弃的时候，一阵清脆的流水声吸引了他的注意。跟随声音，他意外发现了一处隐蔽的山涧。这里的景色美得让他忘记了迷路的恐惧：瀑布从高处倾泻而下，水花四溅，阳光透过树叶的缝隙，在水面上洒下斑驳的光影。他感到内心从焦虑到平静的转变，仿佛这片自然奇观激发了他内在的勇敢和智慧。

在与山涧美景相伴的过程中，李明注意到了一群奇特的生物。它们有着透明的翅膀，身体呈现出斑斓的色彩，就像是从梦境中走出来的精灵。起初，这些生物对他保持

In [28]:
wp.print_text()

《山涧奇遇》
总字数要求约1000字；已完成。
扩写指南 >>> 故事背景设置在一片神秘的山林中，主人公在山中迷路，意外发现一处山涧，并在那里遇到了奇特的生物。主人公需具备勇敢和智慧的品质。建议在故事中包含对山涧美景的描绘，以及与生物之间的互动和感悟。

1 迷路与发现
 李明，一个充满好奇心和冒险精神的青年，踏入了被称为「遗忘之地」的神秘山林。他本想一探究竟，却未曾想到自己会在这片绿意盎然中迷失方向。他在森林中穿梭，试图寻找出路，但四周的景象似乎在不断变幻，让他愈发困惑。就在李明开始感到焦虑时，他意外地走到了一处山涧边。眼前的景色让他惊叹：清澈的溪流在阳光下闪烁，瀑布如丝般顺滑，周围的花草散发着迷人的香气。李明的心中涌起一股宁静，他感到自己的勇敢和智慧在这片自然奇观中得到启发。
2 与生物的互动
 李明在神秘的山林中迷路了。起初，他焦虑不安，四周是茂密的树林和蜿蜒的小径，让他辨不清方向。但就在他几乎要放弃的时候，一阵清脆的流水声吸引了他的注意。跟随声音，他意外发现了一处隐蔽的山涧。这里的景色美得让他忘记了迷路的恐惧：瀑布从高处倾泻而下，水花四溅，阳光透过树叶的缝隙，在水面上洒下斑驳的光影。他感到内心从焦虑到平静的转变，仿佛这片自然奇观激发了他内在的勇敢和智慧。

在与山涧美景相伴的过程中，李明注意到了一群奇特的生物。它们有着透明的翅膀，身体呈现出斑斓的色彩，就像是从梦境中走出来的精灵。起初，这些生物对他保持警惕，但李明没有退缩，他试图用各种方式与它们沟通。他模仿它们的动作，轻声细语地与它们交谈，甚至尝试通过肢体语言表达自己的善意。随着时间的推移，生物们开始放松警惕，与李明展开互动。

在与这些生物的互动中，李明学会了尊重和耐心。他发现，这些生物对于环境的变化极为敏感，它们能通过微妙的信号传递信息。李明开始反思人类社会中的沟通方式，意识到在自然面前，人类还有许多需要学习的地方。他的勇敢不仅体现在面对未知时的勇气，更在于他愿意放下人类的优越感，去理解和适应这些生物的习性。

这次奇遇让李明深刻地领悟到，勇敢和智慧并不仅仅是人类的专利，它们是所有生物在生存斗争中逐渐形成的品质。他开始思考如何将这种与自然和谐相处的方式应用到日常生活中，如何在人类社会中也展现出这样的尊重和理解。


## 全自动生成：更换为GPT4

In [29]:
from langchain_openai import ChatOpenAI

# 使用 OpenAI/ChatGPT 推理
wp = WritingTask(task_mode="auto")
wp.run(llm = ChatOpenAI(model_name="gpt-4-turbo"))

Task Mode: auto
Focus from: root@input
-------------------- root@input --------------------
-------------------- Todos --------------------
<root> None字以内 | 《None》



👤:  我老婆今天跟我生气了，一起生活了20多年，我想哄哄她，帮我写一封道歉信


-------------------- AI for root --------------------
```json
{
    "类型": "root",
    "总字数要求": 400,
    "标题名称": "道歉信",
    "扩写指南": "写作风格应该诚恳、真诚，表达出内心的歉意和对她的重视。可以回顾过去的美好时光，承认自己的错误，并表达出改正的决心。建议在信中适当表达对她的爱和感激，以及对未来更好生活的期许。"
}
```
-------------------- root@output --------------------
-------------------- Todos --------------------
<root> 400字以内 | 《道歉信》
-------------------- Move To: root@output --------------------
👤[auto]:  请帮我扩写《道歉信》, 字数不超过400字，扩写依据为：写作风格应该诚恳、真诚，表达出内心的歉意和对她的重视。可以回顾过去的美好时光，承认自己的错误，并表达出改正的决心。建议在信中适当表达对她的爱和感激，以及对未来更好生活的期许。
-------------------- AI for root --------------------
```json
{
    "类型": "paragraph",
    "标题名称": "道歉信",
    "详细内容": "亲爱的（妻子的姓名），我写这封信是因为我深深地意识到我伤害了你，我感到十分抱歉。我知道我所犯下的错误给你带来了痛苦和失望，而这一切都是我不应该让你承受的。回顾过去的日子，我们一起经历了太多美好的时光，那些幸福的回忆永远珍藏在我心中。我不想因为我的过失而破坏我们的感情，更不愿看到你因此受到伤害。

我承认，我在某些时候没有站在你的角度去考虑问题，我对你的体贴和理解不够，这是我的疏忽。但我保证，我会从这次错误中吸取教训，真诚地改正自己的不足。我会更加用心地倾听你的心声，更加关注你的感受，让我们的关系更加美好和稳固。

我深深地爱着你，你是我的全部。我感激你一直以来的支持和包容，没有你，我的生活将是一片空白。我希望我们能够一起面对生活中的挑战，共同创造更美好的未来。我发誓会为了我

In [11]:
wp.print_lines()

1 风雨之夜的到来
在一个暴风雨的夜晚，约翰和汤姆决心探索镇上那栋有名的老房子。这座房子因其神秘的过去和诡异的传说而闻名。据说，几十年前，一位富有的商人在此神秘失踪，自那以后，无数的探险者和好奇者试图揭开房子的秘密，却无一例外地被各种怪异的事件和不幸的命运所阻挠。约翰和汤姆对超自然现象充满了兴趣，他们相信，通过他们的智慧和勇气，可以揭示这座老房子隐藏的真相。

他们的征途并不顺利。天色逐渐暗沉，狂风大作，雷声隆隆，仿佛大自然也在警告他们不要前进。当他们接近房子时，几个当地人劝他们回头，告诉他们许多人因好奇心而遭受不幸。但约翰和汤姆的决心未被动摇，他们相信这是一次生命中难得的冒险，即使是最恶劣的天气和最棘手的警告也阻挡不了他们的脚步。

2 秘密的低语声


2.1 进入老房子
在风雨交加的夜晚，约翰和汤姆站在那座传说中的老房子前，怀着忐忑不安的心情，他们推开了那扇沉重的前门。门吱嘎作响，仿佛在抗议着这对未被邀请的客人。一进入，一股阴冷及潮湿的空气迎面扑来，使人不寒而栗。房内光线昏暗，只有外面闪电的光芒偶尔透进来，照亮了房间里陈旧的家具和满墙的蛛网。两人试图适应这光线，四处观望，内心的恐惧与好奇心在此刻交织在一起。

2.2 低语声的发现
当约翰和汤姆勇敢地深入老房子的阴暗走廊时，一阵低沉而不连续的低语声突然在空气中回荡。这声音似乎从无处而来，却又无处不在，使得两人不由自主地停下脚步，屏息凝听。他们的眼神交换了一个忐忑的眼色，相互点头后，悄无声息地沿着声音的大致方向前进。汤姆轻轻地把耳朵贴近发霉的墙壁，试图捕捉那若有若无的低语，而约翰则试图通过手电筒的微弱光线，寻找可能的声源。在这种未知与恐惧的驱使下，他们小心翼翼地一步步接近那神秘的低语声。

2.3 追踪与恐惧
随着每一步深入，房子的古老木地板在约翰和汤姆的脚下吱吱作响。隐约的低语声引导他们穿过长长的走廊，每当他们靠近，一阵风似乎从窗外掠过，使得厚重的窗帘无声无息地摇晃。这些动静让房间笼罩在一种不安的气氛中，他们的心跳加速，恐惧与好奇心在胸中交织，推动他们继续前行，尽管内心深处不断有声音告诫他们退后。

3 揭开真相的时刻
最终，约翰和汤姆在房子的最深处，发现了低语声的来源。他们不禁停下脚步，目瞪口呆地望着眼前的场景。在昏暗的灯光下，一张古老的书桌上摆放着一个古怪的箱子，发出微弱的低语声。随着他们的靠近，低语声渐渐

In [10]:
print(wp.root_content.get_outlines())

1 第一章：重名的阴影 
介绍主要人物和背景，发现小说中的名字与现实中的同学相同，开始发生第一个意外事件。
1.1 开篇介绍 
故事背景：静枫中学，主要人物：李晓风，赵瑞，林雨晨，王子豪。
1.2 日常变奏 
主要人物李晓风在图书馆发现一本小说《死亡名单》，里面的角色名字与现实中的同学一模一样，引发了一连串的不安与怀疑。
1.3 阴影初现 
第一个意外事件：王子豪在打扫卫生时，无缘无故从楼梯上摔下，头部受创。这一事件引发主要人物们的不安与怀疑。
1.4 人心惶惶 
意外事件后，主要人物和周围同学都感受到了巨大的心理压力和恐惧，整个学校的氛围变得紧张不安。
2 第二章：恐惧蔓延 
角色们开始感到不安，第二个和第三个重名的同学也遭遇不幸，大家陷入恐慌。
2.1 2.1 不祥的征兆 
第二个重名同学赵瑞频繁遭遇诡异事件，李晓风和林雨晨对此感到担忧和恐惧，担心他会成为下一个不幸者。
2.2 2.2 恐慌的蔓延 
第二个重名同学赵瑞遭遇不幸事件，引发学校内恐慌情绪蔓延，同学们纷纷避开与小说角色重名的人，学校氛围变得沉重。主要人物们意识到必须找到解决方法。
2.3 2.3 灾难的连续 
第三个重名同学林雨晨在类似情况下遭遇不幸，导致主要角色们的恐慌达到顶点。人物：林雨晨，李晓风，王子豪。背景：静枫中学。设定：小说《死亡名单》中的角色与现实中的同学重名，引发连续不幸事件。
2.4 2.4 陷入混乱 
静枫中学的恐慌氛围，主要人物调查诅咒但陷入混乱，其他同学避开他们，调查变得更加困难。
3 第三章：揭秘行动 
主要角色决定调查这一系列事件的背后原因，发现小说的存在与一个古老的诅咒有关。
3.1 3.1 调查启动 
主要人物李晓风、赵瑞、林雨晨在图书馆展开首次调查会议，制定调查计划，分工合作，开始揭秘行动。涉及实体：李晓风、赵瑞、林雨晨、静枫中学图书馆。设定：揭开《死亡名单》背后的谜团，分工合作进行调查。
3.2 3.2 线索搜集 
人物：李晓风、赵瑞、林雨晨。地点：静枫中学图书馆。情节：角色们搜集到小说《死亡名单》与古老诅咒的联系。背景设定：静枫中学图书馆藏有关于诅咒的线索，学校可能隐藏着揭开诅咒的秘密。
3.3 3.3 诅咒揭秘 
在本段中，主要角色们在图书馆的古老资料中发现了《死亡名单》小说与一个古老诅咒的联系。这个诅咒源自几百年前一位被背叛的术士，诅咒的名字会传递并给继承者带

# 测试 WritingTask
## ask_user

In [3]:
from langchain_chinese import WritingTask

# 使用默认的智谱AI推理
wp = WritingTask(task_mode="askme")
wp.ask_user("ok")

(None, None, 'ok', '')

In [6]:
wp.ask_user("1@ask 帮我按两个段落")

('1', '1', 'ask', '帮我按两个段落')

In [5]:
wp.ask_user("START@ask 帮我按两个段落")

('START', 0, 'ask', '帮我按两个段落')

In [7]:
wp.ask_user("ask 帮我按两个段落")

(None, None, 'ask', '帮我按两个段落')

# 测试 TreeContent

In [8]:
from langchain_chinese import TreeContent

root = TreeContent()
a = root.add_item(title="ABC")
b = root.add_item(title="XYZ")
print(a.id, a.title)
print(b.id, b.title)
print(root.id)
print(a.add_item().id)
print(b.add_item().id)
print(b.add_item().root.id)


1 ABC
2 XYZ
0
3
4
0


In [13]:
root.get_item_by_id("4").id

4

In [14]:
root.get_item_by_id("3").id

3